In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats # For statistical tests later

# Set plot style
sns.set_style("whitegrid")
%matplotlib inline

# --- 1. Data Loading ---
# NOTE: Download the file and place it in a 'data' folder or reference the path directly
# Assuming you named the downloaded file 'historical_claims_data.csv'
try:
    df = pd.read_csv('../data/MachineLearningRating_v3.txt')
except FileNotFoundError:
    print("Please download the data and ensure the path is correct.")
    
print(f"Dataset loaded with {df.shape[0]} rows and {df.shape[1]} columns.")

In [ ]:
# Descriptive Statistics (KPI Check: Variability)
print("\n--- Descriptive Statistics for Financials ---")
financial_cols = ['TotalPremium', 'TotalClaims', 'CustomValueEstimate', 'SumInsured', 'CalculatedPremiumPerTerm']
print(df[financial_cols].describe().T)

# Data Structure Review (KPI Check: Data Structure)
print("\n--- Data Structure (Dtypes) ---")
print(df.info())

# Initial Data Preview
print("\n--- Data Head ---")
print(df.head())

In [ ]:
# Missing Values Check (KPI Check: Data Quality Assessment)
missing_data = df.isnull().sum()
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)
print("\n--- Missing Data Report ---")
print(missing_data[missing_data > 0])

# Example: If 'Cylinders' has missing values, investigate why and whether to impute/drop.
# For now, we will proceed assuming necessary cleaning happens before modeling.

In [ ]:
# Distribution of TotalPremium (KPI Check: Univariate Analysis)
plt.figure(figsize=(8, 6))
sns.histplot(df['TotalPremium'], bins=50, kde=True)
plt.title('Distribution of Total Premium')
plt.xlabel('Total Premium')
plt.xlim(0, df['TotalPremium'].quantile(0.99)) # Capping x-axis to 99th percentile for clarity
plt.show()

# Distribution of Rating (Categorical Example)
plt.figure(figsize=(8, 6))
sns.countplot(y='Province', data=df, order=df['Province'].value_counts().index)
plt.title('Policy Count by Province')
plt.xlabel('Policy Count')
plt.ylabel('Province')
plt.show()

In [ ]:
# --- Calculate Loss Ratio (LR) ---
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium']
# Cap LR outliers for visualization (e.g., policy with tiny premium but high claim)
df['LossRatio_Capped'] = df['LossRatio'].clip(upper=3) 

# Guiding Question: How does LR vary by Province? (Creative Plot 1)
plt.figure(figsize=(10, 7))
sns.boxplot(x='LossRatio_Capped', y='Province', data=df)
plt.title('Loss Ratio Distribution by Province (Capped at 3)')
plt.xlabel('Loss Ratio')
plt.ylabel('Province')
plt.axvline(x=1.0, color='r', linestyle='--', label='Breakeven LR (1.0)')
plt.legend()
plt.show()
# 

# Guiding Question: How does LR vary by VehicleType? (Creative Plot 2)
plt.figure(figsize=(8, 6))
sns.barplot(x='VehicleType', y='LossRatio', data=df.groupby('VehicleType')['LossRatio'].mean().reset_index())
plt.title('Average Loss Ratio by Vehicle Type')
plt.xticks(rotation=45, ha='right')
plt.ylabel('Average Loss Ratio')
plt.xlabel('Vehicle Type')
plt.show()

# Guiding Question: Are there temporal trends? (Creative Plot 3)
df['TransactionDate'] = pd.to_datetime(df['The transaction date'])
df['TransactionMonth'] = df['TransactionDate'].dt.to_period('M')

monthly_summary = df.groupby('TransactionMonth')[['TotalClaims', 'TotalPremium']].sum().reset_index()
monthly_summary['MonthlyLR'] = monthly_summary['TotalClaims'] / monthly_summary['TotalPremium']

monthly_summary['TransactionMonth'] = monthly_summary['TransactionMonth'].astype(str)

plt.figure(figsize=(12, 6))
sns.lineplot(x='TransactionMonth', y='MonthlyLR', data=monthly_summary, marker='o')
plt.title('Monthly Portfolio Loss Ratio Trend (Feb 2014 - Aug 2015)')
plt.xlabel('Transaction Month')
plt.ylabel('Monthly Loss Ratio')
plt.xticks(rotation=45)
plt.axhline(y=1.0, color='r', linestyle='--', label='Breakeven LR')
plt.legend()
plt.show()

In [ ]:
# Outlier Detection Example
plt.figure(figsize=(8, 6))
sns.boxplot(y=df['TotalClaims'])
plt.title('Box Plot for Total Claims (Outlier Detection)')
plt.show()
# Investigate high outliers in TotalClaims
high_claims_outliers = df[df['TotalClaims'] > df['TotalClaims'].quantile(0.99)]
print(high_claims_outliers.head())